This notebook is used to organize the experiments. If you just want the best models found, you can refer to the `3_model_training` notebook.

In [52]:
import os

import mlflow
from sklearn.model_selection import train_test_split
from mlflow.models import infer_signature
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score, log_loss
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from functools import partial

os.sys.path.append(os.path.abspath('../src'))
from data import loader
from data import preprocessor

mapname = 'Town01'

In [53]:
def create_and_train_model(model, args):
    """
    Function to create and train a model with given parameters.

    Args:
        model: The model to be trained.
        args: Arguments for training the model.

    Returns:
        result: The result of the training process.
    """
    result = {}
    # Train model with current hyperparameters
    print(f"Params: {args}")
    md = model(**args)
    md.fit(X_train, y_train)
    # Predict on the validation set
    y_pred = md.predict(X_val)
    # Log training results
    result["f1_score"] = f1_score(y_val, y_pred, average='weighted')
    result["accuracy"] = accuracy_score(y_val, y_pred)
    result["recall"] = recall_score(y_val, y_pred, average='weighted')
    result["precision"] = precision_score(y_val, y_pred, average='weighted')
    result["model"] = md

    return result

In [54]:
def objective(params, model_name):
    """
    Objective function for hyperparameter optimization.
    This function will be called by Hyperopt for each trial.
    """
    with mlflow.start_run(nested=True, run_name=f"{model_name}_{params}"):
        # Log hyperparameters being tested
        mlflow.log_params(params)

        # Train model with current hyperparameters
        result = create_and_train_model(
            model,
            args=params
        )

        # Log training results
        mlflow.log_metrics(
            {
                "accuracy": result["accuracy"],
                "recall": result["recall"],
                "precision": result["precision"],
                "f1_score": result["f1_score"],
            }
        )

        # Log the trained model
        mlflow.sklearn.log_model(
            result["model"], name=model_name, signature=signature)

        # Return loss for Hyperopt (it minimizes)
        return {"loss": 1 - result["f1_score"], "status": STATUS_OK}

In [55]:
def run_experiments(run_name, max_evals, search_space, data_name, model_name=None):
    # Create or set experiment

    print(
        f"This will run {max_evals} trials to find optimal hyperparameters...")

    with mlflow.start_run(run_name=run_name):

        mlflow.set_tag("model_type", model_name)

        # Log experiment metadata
        mlflow.log_params(
            {
                "optimization_method": "Tree-structured Parzen Estimator (TPE)",
                "max_evaluations": max_evals,
                "objective_metric": "log_loss",
                "dataset": data_name,
                "model_type": model_name
            }
        )

        # Run optimization
        trials = Trials()
        objective_with_name = partial(objective, model_name=model_name)

        best_params = fmin(
            fn=objective_with_name,
            space=search_space,
            algo=tpe.suggest,
            max_evals=max_evals,
            trials=trials,
            verbose=True,
        )

        # Find and log best results
        best_trial = min(trials.results, key=lambda x: x["loss"])
        best_f1_score  = best_trial["loss"]

        # Log optimization results
        mlflow.log_params(best_params)
        mlflow.log_metrics(
            {
                "best_f1_score ": best_f1_score ,
                "total_trials": len(trials.trials),
                "optimization_completed": 1,
            }
        )

    return best_f1_score 


In [56]:
client = mlflow.tracking.MlflowClient("http://127.0.0.1:8080")
mlflow.set_tracking_uri("http://127.0.0.1:8080")

## Preparing the Data

In [57]:
data_path = '../data'
uah_training = pd.read_csv(f'{data_path}/base/training_set_uah.csv')
uah_validation = pd.read_csv(f'{data_path}/base/validation_set_uah.csv')

carla_fixed = pd.read_csv(f'{data_path}/merged/{mapname}/carla/carla_fixed.csv').drop(columns=['origin'])
carla_llm = pd.read_csv(f'{data_path}/merged/{mapname}/carla/carla_llm.csv').drop(columns=['origin'])

sumo_fixed = pd.read_csv(f'{data_path}/merged/{mapname}/sumo/sumo_fixed.csv').drop(columns=['origin'])
sumo_llm = pd.read_csv(f'{data_path}/merged/{mapname}/sumo/sumo_llm.csv').drop(columns=['origin'])

carla_uah_fixed = pd.read_csv(f'{data_path}/merged/{mapname}/carla_uah/carla_uah_fixed.csv').drop(columns=['origin'])
carla_uah_llm = pd.read_csv(f'{data_path}/merged/{mapname}/carla_uah/carla_uah_llm.csv').drop(columns=['origin'])

sumo_uah_fixed = pd.read_csv(f'{data_path}/merged/{mapname}/sumo_uah/sumo_uah_fixed.csv').drop(columns=['origin'])
sumo_uah_llm = pd.read_csv(f'{data_path}/merged/{mapname}/sumo_uah/sumo_uah_llm.csv').drop(columns=['origin'])

Applying sliding windows to UAH, SUMO and CARLA full data

In [58]:
window_size = 10
step_size = 5
one_hot_keys = {
    'normal': 0,
    'aggressive': 1
}

In [59]:
# SUMO
X_sumo_fixed, y_sumo_fixed = preprocessor.sliding_windows(sumo_fixed, window_size=window_size, step_size=step_size)
X_sumo_llm, y_sumo_llm = preprocessor.sliding_windows(sumo_llm, window_size=window_size, step_size=step_size)

# CARLA
X_carla_fixed, y_carla_fixed = preprocessor.sliding_windows(carla_fixed, window_size=window_size, step_size=step_size)
X_carla_llm, y_carla_llm = preprocessor.sliding_windows(carla_llm, window_size=window_size, step_size=step_size)

Defining the search space for each model

In [64]:
max_evals = 5

search_space_rf = {
    "n_estimators": hp.choice("n_estimators", [20, 50, 100, 200, 500]),
    "max_depth": hp.choice("max_depth", [None, 5, 10, 20, 50]),
    "random_state": 42,
}
search_space_svc = {
    "C": hp.choice("C", [0.01, 0.05, 0.1, 0.5, 1]),  # Regularization parameter
    "kernel": hp.choice("kernel", ["rbf"]),  # Kernel type
    "gamma": hp.choice("gamma", [0.001, 0.01, 0.1, 1, 10]),  # Kernel coefficient for 'rbf', 'poly', and 'sigmoid'
}
search_space_svc_linear = {
    "C": hp.choice("C", [10**-4, 10**-3, 10**-2, 0.1, 1])  # Regularization parameter
}
search_space_xgb = {
    "n_estimators": hp.choice("n_estimators", [200, 500, 700, 1000]),
    "max_depth": hp.choice("max_depth", [None, 5, 10]),
    "learning_rate": hp.choice("learning_rate", [0.001, 0.01, 0.1, 0.2, 0.3, 0.4, 0.5]),  # Learning rate
    # "subsample": hp.uniform("subsample", 0.5, 1.0),
    # "colsample_bytree": hp.uniform("colsample_bytree", 0.5, 1.0),
    "random_state": 42,
}

## Real Only

In [65]:
# Provide an Experiment description that will appear in the UI
experiment_description = (
    "Experiment to train models on the UAH driveset."
)

# Provide searchable tags that define characteristics of the Runs that
# will be in this Experiment
experiment_tags = {
    "project_name": "driver-behavior-prediction",
    "mlflow.note.content": experiment_description,
}

# Create the Experiment, providing a unique name
try:
    driver_behavior_experiment = client.create_experiment(
        name="Driver_Behavior_Models_UAH", tags=experiment_tags
    )
except mlflow.exceptions.RestException as e:
    print(f"Experiment already exists: {e}")

In [66]:
columns_to_keep = ['acc_x', 'acc_y', 'acc_z', 'gyro_x', 'gyro_y', 'gyro_z', 'angle', 'speed', 'label']

X_train, y_train = preprocessor.sliding_windows(uah_training[columns_to_keep], window_size=window_size, step_size=step_size)
X_val, y_val = preprocessor.sliding_windows(uah_validation[columns_to_keep], window_size=window_size, step_size=step_size)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

y_train = preprocessor.one_hot_encode(y_train, one_hot_keys=one_hot_keys)
y_val = preprocessor.one_hot_encode(y_val, one_hot_keys=one_hot_keys)

signature = infer_signature(X_train, y_train)
mlflow.set_experiment('Driver_Behavior_Models_UAH')
data_name = 'UAH'

In [67]:
model = RandomForestClassifier
run_experiments('rf-sweep', max_evals=max_evals, search_space=search_space_rf, data_name='UAH', model_name = 'RandomForestClassifier')
model = SVC
run_experiments('svc-sweep', max_evals=max_evals, search_space=search_space_svc, data_name='UAH', model_name = 'SVC')
model = LinearSVC
run_experiments('svc-linear-sweep', max_evals=max_evals, search_space=search_space_svc_linear, data_name='UAH', model_name = 'LinearSVC')
model = xgb.XGBClassifier
run_experiments('xgb-sweep', max_evals=max_evals, search_space=search_space_xgb, data_name='UAH', model_name = 'XGBClassifier')

This will run 5 trials to find optimal hyperparameters...
Params: {'max_depth': 5, 'n_estimators': 200, 'random_state': 42}
🏃 View run RandomForestClassifier_{'max_depth': 5, 'n_estimators': 200, 'random_state': 42} at: http://127.0.0.1:8080/#/experiments/500441082922571946/runs/b392eca2c357454c9c8e95a0d3128cfc

🧪 View experiment at: http://127.0.0.1:8080/#/experiments/500441082922571946

Params: {'max_depth': 20, 'n_estimators': 20, 'random_state': 42}              
🏃 View run RandomForestClassifier_{'max_depth': 20, 'n_estimators': 20, 'random_state': 42} at: http://127.0.0.1:8080/#/experiments/500441082922571946/runs/c4ddbe2fe20140efb4ecdd501c1d1d20

🧪 View experiment at: http://127.0.0.1:8080/#/experiments/500441082922571946  

Params: {'max_depth': 50, 'n_estimators': 100, 'random_state': 42}              
🏃 View run RandomForestClassifier_{'max_depth': 50, 'n_estimators': 100, 'random_state': 42} at: http://127.0.0.1:8080/#/experiments/500441082922571946/runs/207b558e069a4e6985ce

/home/renan/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



🏃 View run SVC_{'C': 1, 'gamma': 10, 'kernel': 'rbf'} at: http://127.0.0.1:8080/#/experiments/500441082922571946/runs/1d617889bf1a40de9460497ec476da24

🧪 View experiment at: http://127.0.0.1:8080/#/experiments/500441082922571946  

Params: {'C': 0.5, 'gamma': 10, 'kernel': 'rbf'}                               
 40%|████      | 2/5 [01:39<02:46, 55.56s/trial, best loss: 0.4164731436766872]

/home/renan/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



🏃 View run SVC_{'C': 0.5, 'gamma': 10, 'kernel': 'rbf'} at: http://127.0.0.1:8080/#/experiments/500441082922571946/runs/a42747afdb4d4a67a6130cc249611c05

🧪 View experiment at: http://127.0.0.1:8080/#/experiments/500441082922571946  

Params: {'C': 0.05, 'gamma': 0.01, 'kernel': 'rbf'}                            
🏃 View run SVC_{'C': 0.05, 'gamma': 0.01, 'kernel': 'rbf'} at: http://127.0.0.1:8080/#/experiments/500441082922571946/runs/040c08b5aaab421687848f2543993072

🧪 View experiment at: http://127.0.0.1:8080/#/experiments/500441082922571946  

Params: {'C': 1, 'gamma': 0.1, 'kernel': 'rbf'}                                 
🏃 View run SVC_{'C': 1, 'gamma': 0.1, 'kernel': 'rbf'} at: http://127.0.0.1:8080/#/experiments/500441082922571946/runs/fc2016ad4a8445829afd78fda6b0a537

🧪 View experiment at: http://127.0.0.1:8080/#/experiments/500441082922571946   

100%|██████████| 5/5 [04:06<00:00, 49.25s/trial, best loss: 0.35791934767986433]
🏃 View run svc-sweep at: http://127.0.0.1:8080/#/expe

/home/renan/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



🏃 View run XGBClassifier_{'learning_rate': 0.001, 'max_depth': 5, 'n_estimators': 200, 'random_state': 42} at: http://127.0.0.1:8080/#/experiments/500441082922571946/runs/d841e73074de4b8595644f2c2d5d075a

🧪 View experiment at: http://127.0.0.1:8080/#/experiments/500441082922571946  

Params: {'learning_rate': 0.3, 'max_depth': 5, 'n_estimators': 500, 'random_state': 42}
🏃 View run XGBClassifier_{'learning_rate': 0.3, 'max_depth': 5, 'n_estimators': 500, 'random_state': 42} at: http://127.0.0.1:8080/#/experiments/500441082922571946/runs/06ae53150e4d47febf27f159c3851bab

🧪 View experiment at: http://127.0.0.1:8080/#/experiments/500441082922571946  

Params: {'learning_rate': 0.01, 'max_depth': None, 'n_estimators': 200, 'random_state': 42}
🏃 View run XGBClassifier_{'learning_rate': 0.01, 'max_depth': None, 'n_estimators': 200, 'random_state': 42} at: http://127.0.0.1:8080/#/experiments/500441082922571946/runs/eb5dcbddf0a04b9399d0e67b28b8df6d

🧪 View experiment at: http://127.0.0.1:8080/#

0.4129187150726529

## SUMO Only

In [ ]:
# Getting the validation for SUMO variables
columns_to_keep = ['acc_x', 'acc_y', 'angle', 'speed', 'label']
X_val_base, y_val = preprocessor.sliding_windows(uah_validation[columns_to_keep], window_size=window_size, step_size=step_size)
y_val = preprocessor.one_hot_encode(y_val, one_hot_keys=one_hot_keys)

### Fixed

In [77]:
# Provide an Experiment description that will appear in the UI
experiment_description = (
    "Experiment to train models on the SUMO Fixed dataset."
)

# Provide searchable tags that define characteristics of the Runs that
# will be in this Experiment
experiment_tags = {
    "project_name": "driver-behavior-prediction",
    "mlflow.note.content": experiment_description,
}

# Create the Experiment, providing a unique name
try:
    driver_behavior_experiment = client.create_experiment(
        name="Driver_Behavior_Models_SUMO_Fixed", tags=experiment_tags
    )
except mlflow.exceptions.RestException as e:
    print(f"Experiment already exists: {e}")

Experiment already exists: RESOURCE_ALREADY_EXISTS: Experiment 'Driver_Behavior_Models_SUMO_Fixed' already exists.


In [ ]:
X_train, y_train = X_sumo_fixed, y_sumo_fixed

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val_base)
y_train = preprocessor.one_hot_encode(y_train, one_hot_keys=one_hot_keys)

signature = infer_signature(X_train, y_train)
mlflow.set_experiment('Driver_Behavior_Models_SUMO_Fixed')
data_name = 'SUMO_Fixed'

In [80]:
model = RandomForestClassifier
run_experiments('rf-sweep', max_evals=max_evals, search_space=search_space_rf, data_name=data_name)
# model = SVC
# run_experiments('svc-sweep', max_evals=max_evals, search_space=search_space_svc, data_name=data_name)
# model = xgb.XGBClassifier
# run_experiments('xgb-sweep', max_evals=max_evals, search_space=search_space_xgb, data_name=data_name)

This will run 5 trials to find optimal hyperparameters...
  0%|          | 0/5 [00:00<?, ?trial/s, best loss=?]

🏃 View run monumental-stoat-40 at: http://127.0.0.1:8080/#/experiments/105017844406316764/runs/3a131b7d728b4f80b8a231be3526bd75

🧪 View experiment at: http://127.0.0.1:8080/#/experiments/105017844406316764

🏃 View run gifted-sheep-802 at: http://127.0.0.1:8080/#/experiments/105017844406316764/runs/68a9772c546644e5a738acd504b17848

🧪 View experiment at: http://127.0.0.1:8080/#/experiments/105017844406316764  

🏃 View run respected-bee-475 at: http://127.0.0.1:8080/#/experiments/105017844406316764/runs/f3f92d14006246d0badf7480c11d68a9

🧪 View experiment at: http://127.0.0.1:8080/#/experiments/105017844406316764  

🏃 View run bold-bat-19 at: http://127.0.0.1:8080/#/experiments/105017844406316764/runs/2f4a0acc2a5c404b9a383dc947d55733

🧪 View experiment at: http://127.0.0.1:8080/#/experiments/105017844406316764  

🏃 View run suave-horse-677 at: http://127.0.0.1:8080/#/experiments/105017844406316764/runs/432bfa0332084cadaf586785aec27135

🧪 View experiment at: http://127.0.0.1:8080/#/experime

18.033368027415726

### LLM

In [81]:
# Provide an Experiment description that will appear in the UI
experiment_description = (
    "Experiment to train models on the SUMO LLM dataset."
)

# Provide searchable tags that define characteristics of the Runs that
# will be in this Experiment
experiment_tags = {
    "project_name": "driver-behavior-prediction",
    "mlflow.note.content": experiment_description,
}

# Create the Experiment, providing a unique name
try:
    driver_behavior_experiment = client.create_experiment(
        name="Driver_Behavior_Models_SUMO_LLM", tags=experiment_tags
    )
except mlflow.exceptions.RestException as e:
    print(f"Experiment already exists: {e}")

In [ ]:
X_train, y_train = X_sumo_llm, y_sumo_llm

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val_base)
y_train = preprocessor.one_hot_encode(y_train, one_hot_keys=one_hot_keys)

mlflow.set_experiment('Driver_Behavior_Models_SUMO_LLM')
data_name = 'SUMO_LLM'

In [83]:
model = RandomForestClassifier
run_experiments('rf-sweep', max_evals=max_evals, search_space=search_space_rf, data_name=data_name)
# model = SVC
# run_experiments('svc-sweep', max_evals=max_evals, search_space=search_space_svc, data_name=data_name)
# model = xgb.XGBClassifier
# run_experiments('xgb-sweep', max_evals=max_evals, search_space=search_space_xgb, data_name=data_name)

This will run 5 trials to find optimal hyperparameters...
  0%|          | 0/5 [00:00<?, ?trial/s, best loss=?]

/home/renan/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



🏃 View run nimble-skink-889 at: http://127.0.0.1:8080/#/experiments/207753888438448124/runs/910d6ee714d3492ab08a41965e33842f

🧪 View experiment at: http://127.0.0.1:8080/#/experiments/207753888438448124

 20%|██        | 1/5 [00:03<00:13,  3.49s/trial, best loss: 8.625255231143939]

/home/renan/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



🏃 View run nervous-crab-437 at: http://127.0.0.1:8080/#/experiments/207753888438448124/runs/995c493c61804a5e9a2c55065ecdb1c4

🧪 View experiment at: http://127.0.0.1:8080/#/experiments/207753888438448124 

🏃 View run capricious-asp-388 at: http://127.0.0.1:8080/#/experiments/207753888438448124/runs/8527f47105a84a1e856a64e71ba363a1

🧪 View experiment at: http://127.0.0.1:8080/#/experiments/207753888438448124

 60%|██████    | 3/5 [00:09<00:05,  2.83s/trial, best loss: 6.544624351583295]

/home/renan/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



🏃 View run learned-snake-91 at: http://127.0.0.1:8080/#/experiments/207753888438448124/runs/4e25bae572a140b69c11dc0ae398d80c

🧪 View experiment at: http://127.0.0.1:8080/#/experiments/207753888438448124 

 80%|████████  | 4/5 [00:12<00:03,  3.18s/trial, best loss: 6.544624351583295]

/home/renan/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



🏃 View run rare-sponge-680 at: http://127.0.0.1:8080/#/experiments/207753888438448124/runs/20a8cd09db5c49a083368375f696e0d5

🧪 View experiment at: http://127.0.0.1:8080/#/experiments/207753888438448124 

100%|██████████| 5/5 [00:17<00:00,  3.42s/trial, best loss: 6.544624351583295]
{'max_depth': np.int64(1), 'n_estimators': np.int64(0)}
🏃 View run rf-sweep at: http://127.0.0.1:8080/#/experiments/207753888438448124/runs/de43b26c616541fd924dab7cbb5bb61a
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/207753888438448124


6.544624351583295

## Carla Only

In [ ]:
# Getting the validation for SUMO variables
columns_to_keep = ['acc_x', 'acc_y', 'acc_z', 'gyro_x', 'gyro_y', 'gyro_z', 'angle', 'label']
X_val, y_val = preprocessor.sliding_windows(uah_validation[columns_to_keep], window_size=window_size, step_size=step_size)
y_val = preprocessor.one_hot_encode(y_val, one_hot_keys=one_hot_keys)

### Fixed

In [ ]:
# Provide an Experiment description that will appear in the UI
experiment_description = (
    "Experiment to train models on the CARLA Fixed dataset."
)

# Provide searchable tags that define characteristics of the Runs that
# will be in this Experiment
experiment_tags = {
    "project_name": "driver-behavior-prediction",
    "mlflow.note.content": experiment_description,
}

# Create the Experiment, providing a unique name
try:
    driver_behavior_experiment = client.create_experiment(
        name="Driver_Behavior_Models_CARLA_Fixed", tags=experiment_tags
    )
except mlflow.exceptions.RestException as e:
    print(f"Experiment already exists: {e}")

In [ ]:
X_train, y_train = X_carla_fixed, y_carla_fixed
y_train = preprocessor.one_hot_encode(y_train, one_hot_keys=one_hot_keys)

mlflow.set_experiment('Driver_Behavior_Models_CARLA_Fixed')
data_name = 'CARLA_Fixed'

In [ ]:
model = RandomForestClassifier
run_experiments('rf-sweep', max_evals=max_evals, search_space=search_space_rf, data_name=data_name)
model = SVC
run_experiments('svc-sweep', max_evals=max_evals, search_space=search_space_svc, data_name=data_name)
model = xgb.XGBClassifier
run_experiments('xgb-sweep', max_evals=max_evals, search_space=search_space_xgb, data_name=data_name)

### LLM

In [ ]:
# Provide an Experiment description that will appear in the UI
experiment_description = (
    "Experiment to train models on the CARLA LLM dataset."
)

# Provide searchable tags that define characteristics of the Runs that
# will be in this Experiment
experiment_tags = {
    "project_name": "driver-behavior-prediction",
    "mlflow.note.content": experiment_description,
}

# Create the Experiment, providing a unique name
try:
    driver_behavior_experiment = client.create_experiment(
        name="Driver_Behavior_Models_CARLA_LLM", tags=experiment_tags
    )
except mlflow.exceptions.RestException as e:
    print(f"Experiment already exists: {e}")

In [ ]:
X_train, y_train = X_carla_llm, y_carla_llm

y_train = preprocessor.one_hot_encode(y_train, one_hot_keys=one_hot_keys)


mlflow.set_experiment('Driver_Behavior_Models_CARLA_LLM')
data_name = 'CARLA_LLM'

In [ ]:
model = RandomForestClassifier
run_experiments('rf-sweep', max_evals=max_evals, search_space=search_space_rf, data_name=data_name)
model = SVC
run_experiments('svc-sweep', max_evals=max_evals, search_space=search_space_svc, data_name=data_name)
model = xgb.XGBClassifier
run_experiments('xgb-sweep', max_evals=max_evals, search_space=search_space_xgb, data_name=data_name)

## Real + SUMO

### Fixed

In [176]:
signature = infer_signature(X_sumo_fixed, y_sumo_fixed)

In [ ]:
# Provide an Experiment description that will appear in the UI
experiment_description = (
    "Experiment to train models on the UAH driveset supplemented by SUMO with fixed parameters."
)

# Provide searchable tags that define characteristics of the Runs that
# will be in this Experiment
experiment_tags = {
    "project_name": "driver-behavior-prediction",
    "mlflow.note.content": experiment_description,
}

# Create the Experiment, providing a unique name
driver_behavior_experiment = client.create_experiment(
    name="Driver_Behavior_Models_UAH_SUMO_fixed", tags=experiment_tags
)

In [ ]:
model = RandomForestClassifier
run_experiments('rf-sweep', max_evals=5, search_space=search_space_rf, data_name='UAH')

## Real + SUMO (LLM)